In [5]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import urllib.request
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
%matplotlib inline

In [6]:
'''NFL FASTR Documentation:
https://www.rdocumentation.org/packages/nflscrapR/versions/1.8.1/topics/scrape_json_play_by_play'''

'NFL FASTR Documentation:\nhttps://www.rdocumentation.org/packages/nflscrapR/versions/1.8.1/topics/scrape_json_play_by_play'

In [7]:
'''Procure Data'''

URL = 'https://github.com/guga31bb/nflfastR-data/blob/master/'
Unzip = '.csv.gz?raw=True'

In [8]:
'''Pull data'''

def pull_PBP_data(YEAR):
    
    data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(YEAR) + Unzip,
                         compression='gzip', low_memory=False)
    
    return data

In [9]:
'''Data frames from last 3 years'''

data_2020 = pd.DataFrame(pull_PBP_data(2020))
data_2019 = pd.DataFrame(pull_PBP_data(2019))
data_2018 = pd.DataFrame(pull_PBP_data(2018))


In [10]:
# Neutral Situation

def Neutral_Situation_Call(DF):
    neutral_situation = DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8 & play_type != 'no_play'")
    return neutral_situation

neutral_situation = Neutral_Situation_Call(data_2020)

In [22]:
# Team List
Team_List = ['ARI',
'ATL',
'BAL',
'BUF',
'CAR',
'CHI',
'CIN',
'CLE',
'DAL',
'DEN',
'DET',
'GB',
'HOU',
'IND',
'JAX',
'KC',
'LA',
'LAC',
'LV',
'MIA',
'MIN',
'NE',
'NO',
'NYG',
'NYJ',
'PHI',
'PIT',
'SEA',
'SF',
'TB',
'TEN',
'WAS']





In [35]:
#CHOOSE TEAM

def Select_Team():
    
    team = input('Input Team Name:')
    
    team = team.upper()

    if team in Team_List:
        return team
    else:
        print('Please input a valid team')

    return team

print(Select_Team())



CHI


In [36]:
## PLAYCALL FREQUENCY BY PLAY TYPE & DOWN

def PlayFreq_ByTeam_Down(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.run_location, Play_filter.run_gap], columns= [Play_filter.shotgun,Play_filter.down] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    elif PlayType == 'pass':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.pass_location, Play_filter.pass_length], columns= [ Play_filter.shotgun, Play_filter.down] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    else:
        print("invalid play type")


    return round(Tab*100,1)

PlayFreq_ByTeam_Down(data_2020,'pass',Team)

<ipython-input-36-d6ce4d753ee5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


shotgun                                         0           1            
down                                          1.0   2.0   1.0   2.0   3.0
posteam play_type pass_location pass_length                              
CHI     pass      left          deep         14.0  11.1   0.0   6.2   3.9
                                short        30.0  33.3  32.7  34.4  21.1
                  middle        deep          2.0  18.5   1.9   3.1   6.6
                                short         8.0   3.7  28.8  15.6  26.3
                  right         deep          8.0   7.4   9.6   4.7   5.3
                                short        38.0  25.9  26.9  35.9  36.8

In [62]:
## PLAYCALL TYPE FREQUENCY BY DOWN

def PlayFreqType_ByTeam_Down(DF, Team):
 
  


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[(neutral_situation['play_type'] == 'pass') | (neutral_situation['play_type'] == 'run') ]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    
    Tab = pd.crosstab([Play_filter.posteam,  Play_filter.play_type], columns= [Play_filter.shotgun,Play_filter.down] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    

    return round(Tab*100,1)

PlayFreqType_ByTeam_Down(data_2020,Team)


<ipython-input-62-c11c0a8a5dc3>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


shotgun               0                  1            
down                1.0   2.0    3.0   1.0   2.0   3.0
posteam play_type                                     
CHI     pass       37.7  34.2    0.0  67.1  68.6  82.8
        run        62.3  65.8  100.0  32.9  31.4  17.2

In [37]:
## EPA BY DOWN & FORMATION


def PlayEPA_ByTeam_Down(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'run_location', 'run_gap'], columns = ['shotgun','down'], values = ['epa', 'yards_gained' ])
    elif PlayType == 'pass':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'pass_location', 'pass_length'], columns = ['shotgun', 'down'], values = ['epa', 'yards_gained', 'air_yards'])
    else:
        print("invalid play type")


    return round(Tab,2)

PlayEPA_ByTeam_Down(data_2020,'run',Team)

<ipython-input-37-e26c5f8815fe>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


epa                                \
shotgun                                    0                 1               
down                                     1.0   2.0   3.0   1.0   2.0   3.0   
posteam play_type run_location run_gap                                       
CHI     run       left         end     -0.23 -0.34  1.18 -0.48  0.70  0.33   
                               guard   -0.53 -0.02 -1.19 -0.36 -0.40  0.49   
                               tackle  -0.12  0.17   NaN  1.38 -0.50   NaN   
                  right        end     -0.04  0.54  1.55 -0.07  0.39  0.86   
                               guard   -0.46  0.18  0.51 -0.45 -0.51 -0.44   
                               tackle  -0.01 -0.00   NaN  0.11 -0.22 -1.83   

                                       yards_gained                           \
shotgun                                           0                 1          
down                                            1.0   2.0  3.0    1.0    2.0   
posteam play_type run_location run_gap                                         
CHI     run       left         end             3.25  2.11  7.0   1.40  11.00   
                               guard           1.33  2.50  0.0   1.00   3.20   
                               tackle          4.18  4.40  NaN  22.00   3.00   
                  right        end             4.24  5.40  7.0   4.60   8.60   
                               guard           1.60  5.33  3.0   1.50   2.25   
                               tackle          3.96  4.09  NaN   5.33   3.67   

                                              
shotgun                                       
down                                     3.0  
posteam play_type run_location run_gap        
CHI     run       left         end      2.00  
                               guard    3.33  
                               tackle    NaN  
                  right        end      6.00  
                               guard    1.50  
                               tackle   0.00

In [45]:
# EPA by Team, Play-type and Down -- HI LEVEL


def EPA_ByTeam_Down(DF, Team):

    Team_table = neutral_situation[neutral_situation['posteam']==Team]
    
    Table_2 = round(pd.pivot_table(Team_table, index= ['posteam','play_type', 'down'], columns = ['shotgun'], values = ['epa', 'yards_gained']),2)

    return pd.DataFrame(Table_2) 

EPA_ByTeam_Down(data_2020,Team)



epa       yards_gained      
shotgun                    0     1            0     1
posteam play_type down                               
CHI     pass      1.0   0.11  0.01         7.13  6.09
                  2.0   0.50 -0.10        10.63  5.16
                  3.0    NaN -0.39          NaN  4.10
        run       1.0  -0.16  0.09         3.43  6.56
                  2.0   0.05  0.03         4.06  6.41
                  3.0   0.09 -0.06         3.00  2.59

In [63]:
## EPA BY DOWN & FORMATION


def PlayEPA_ByTeam_Down_Player(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'run_location', 'run_gap'], columns = ['shotgun','down', 'rusher'], values = ['epa', 'yards_gained' ])
    elif PlayType == 'pass':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'pass_location', 'pass_length'], columns = ['shotgun', 'down'], values = ['epa', 'yards_gained', 'air_yards'])
    else:
        print("invalid play type")


    return round(Tab,2)

PlayEPA_ByTeam_Down_Player(data_2020,'run',Team)

<ipython-input-63-ed5db61b1b33>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


epa                             \
shotgun                                       0                              
down                                        1.0                              
rusher                                 A.Miller A.Pierce A.Robinson C.Kmet   
posteam play_type run_location run_gap                                       
CHI     run       left         end        -0.29      NaN       -0.9    NaN   
                               guard        NaN      NaN        NaN    NaN   
                               tackle       NaN      NaN        NaN    NaN   
                  right        end          NaN    -0.09        NaN  -1.02   
                               guard        NaN      NaN        NaN    NaN   
                               tackle       NaN     0.24        NaN    NaN   

                                                                          \
shotgun                                                                    
down                                                                       
rusher                                 C.Patterson D.Carter D.Montgomery   
posteam play_type run_location run_gap                                     
CHI     run       left         end           -0.25      NaN         0.06   
                               guard         -0.44      NaN        -0.55   
                               tackle        -0.04      NaN        -0.18   
                  right        end            0.30    -0.13        -0.33   
                               guard         -0.34      NaN        -0.48   
                               tackle        -0.31      NaN         0.01   

                                                                  ...  \
shotgun                                                           ...   
down                                                         2.0  ...   
rusher                                 D.Mooney T.Cohen A.Miller  ...   
posteam play_type run_location run_gap                            ...   
CHI     run       left         end          NaN     NaN      0.9  ...   
                               guard        NaN     NaN      NaN  ...   
                               tackle       NaN     NaN      NaN  ...   
                  right        end         0.28     NaN      NaN  ...   
                               guard        NaN     NaN      NaN  ...   
                               tackle       NaN   -0.02      NaN  ...   

                                       yards_gained                           \
shotgun                                           1                            
down                                            1.0         2.0                
rusher                                   M.Trubisky C.Patterson D.Montgomery   
posteam play_type run_location run_gap                                         
CHI     run       left         end              1.0         NaN         5.00   
                               guard            NaN         0.0         4.00   
                               tackle           NaN         NaN         3.00   
                  right        end              8.0        -1.0        10.67   
                               guard            NaN         NaN         2.25   
                               tackle           NaN         NaN         5.00   

                                                                             \
shotgun                                                                       
down                                                        3.0               
rusher                                 N.Foles T.Cohen A.Pierce C.Patterson   
posteam play_type run_location run_gap                                        
CHI     run       left         end         NaN    17.0      NaN         NaN   
                               guard       NaN     NaN      NaN         NaN   
                               tackle      NaN     NaN      NaN         NaN   
                  right        

In [76]:
def PlayFreqType_ByTeam_Down_Player(DF,PlayType,Team):
 
  


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[(neutral_situation['play_type'] == PlayType)]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.down,Play_filter.rusher], columns= [Play_filter.run_location,Play_filter.run_gap] ,values = Play_filter.play_id, aggfunc='count', normalize='index')
    elif PlayType == 'pass':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.down, Play_filter.pass_location,Play_filter.pass_length], columns= [Play_filter.receiver] ,values = Play_filter.play_id, aggfunc='count', normalize='index')
    else: 
        print("invalid play type")



    return round(Tab*100,1)

PlayFreqType_ByTeam_Down_Player(data_2020,'pass',Team)

<ipython-input-76-57ec20aa3888>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


receiver                                A.Miller  A.Robinson  C.Kmet  \
posteam down pass_location pass_length                                 
CHI     1.0  left          deep              0.0        28.6    14.3   
                           short             6.2         9.4    18.8   
             middle        deep              0.0        50.0     0.0   
                           short            26.3        42.1     5.3   
             right         deep              0.0        22.2     0.0   
                           short            13.3        30.0     6.7   
        2.0  left          deep              0.0        42.9     0.0   
                           short             6.7        13.3    10.0   
             middle        deep             14.3        42.9     0.0   
                           short             9.1        36.4     9.1   
             right         deep              0.0        50.0     0.0   
                           short             6.7        26.7     3.3   
        3.0  left          deep             33.3        33.3     0.0   
                           short            12.5        37.5     6.2   
             middle        deep              0.0        80.0     0.0   
                           short            35.0        25.0    10.0   
             right         deep             50.0        25.0     0.0   
                           short            14.8        25.9     3.7   

receiver                                C.Patterson  D.Harris  D.Montgomery  \
posteam down pass_location pass_length                                        
CHI     1.0  left          deep                 0.0       0.0           0.0   
                           short                9.4       0.0          15.6   
             middle        deep                 0.0       0.0           0.0   
                           short               10.5       5.3           5.3   
             right         deep                 0.0       0.0          11.1   
                           short                6.7       3.3           3.3   
        2.0  left          deep                 0.0       0.0           0.0   
                           short               13.3       3.3          10.0   
             middle        deep                 0.0       0.0           0.0   
                           short                9.1       9.1           0.0   
             right         deep                 0.0       0.0           0.0   
                           short                0.0      10.0           3.3   
        3.0  left          deep                 0.0       0.0           0.0   
                           short                6.2       6.2          12.5   
             middle        deep                20.0       0.0           0.0   
                           short                5.0       0.0          10.0   
             right         deep                 0.0       0.0           0.0   
                           short               11.1       0.0          11.1   

receiver                                D.Mooney  J.Graham  J.Wims  L.Miller  \
posteam down pass_location pass_length                                         
CHI     1.0  left          deep             42.9       0.0     0.0       0.0   
                           short            15.6      15.6     3.1       3.1   
             middle        deep              0.0       0.0    50.0       0.0   
                           short             0.0       5.3     0.0       0.0   
             right         deep             33.3       0.0    22.2       0.0   
                           short            13.3      23.3     0.0       0.0   
        2.0  left          deep             42.9      14.3     0.0       0.0   
                           short            26.7      10.0     3.3       0.0   
             middle        deep             42.9       0.0     0.0       0.0   
                           short             9.1      18.2     0.0       0.0   
             right